**참고 사이트**<br>
[첫 번째 참고 사이트](https://blog.naver.com/cygnet3rd/222885102852),
[두 번째 참고 사이트](https://velog.io/@leeug9002/python-selenium%EC%9D%B8%EC%8A%A4%ED%83%80-%ED%81%AC%EB%A1%A4%EB%A7%81),
[세 번째 참고 사이트](https://proefforter.tistory.com/36)

**필요한 패키지 호출**

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

import requests
from re import escape
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import urllib.request
import time
from selenium.webdriver.common.by import By

**데이터 수집하기**
- 아이디, 본문 내용, 해시태그, 작성일자, 좋아요 수, 이미지 URL, 위치정보

In [2]:
def get_content(driver):
    url = driver.current_url
    driver.get(url)
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    #아이디
    ids = soup.select('span._ap3a._aaco._aacw._aacx._aad7._aade')[0].text

    #본문 내용
    try:
        #다른 유저 태그되어있을 시 [0].text가 "및"임
        soup_select = soup.select('span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj')
        select_cnt = len(soup_select)
        content_all = soup_select[select_cnt-1].text
        content = re.sub(r'#\S+', '', content_all).strip()
    except:
        content = ''    #본문 내용 없을 시

    #해시태그
    tags = re.findall(r'#[^\s#,\\]+', content_all)

    #작성일자
    date = soup.select('time.x1p4m5qa')[0]['datetime'][:10]

    #좋아요
    try:
        like = soup.select('span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')[0].text
    except:
        like = 0    #좋아요 없을 시
    
    #이미지
    try:
        img_urls = []

        total_images = int(len(soup.select('div._acnb')))

        for i in range(total_images):
            #슬라이드 후 이미지 로딩 대기
            time.sleep(2)

            #이미지 추출(첫 이미지만 li._acaz[0], 나머지는 [1])
            img_idx = 0 if i == 0 else 1
            try:
                img_cnts = driver.find_elements(By.CSS_SELECTOR, 'li._acaz')
                img = img_cnts[img_idx].find_element(By.TAG_NAME, 'img')
                src = img.get_attribute('src')
                img_urls.append(src)
                #urllib.request.urlretrieve(src, f'{i+1}.jpg')  #이미지 저장장
            except Exception as e:
                print(f'{i+1}번째 이미지 저장 실패: ', e)

            #마지막 이미지 아니면 다음 버튼 클릭
            if i < total_images - 1:
                try:
                    nxt_btn = driver.find_element(By.CSS_SELECTOR, 'div._9zm2')
                    nxt_btn.click()
                except Exception as e:
                    print(f'{i+1}번째에서 다음 버튼 클릭 실패: ', e)
    except:
        img_urls = ''   #이미지 없을 시

    data = [ids, content, tags, date, like, img_urls]
    return data

**크롬 브라우저 열고 인스타그램 접속**

In [3]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = 'https://www.instagram.com'

driver.get(url)

**인스타그램 로그인**
- `By.NAME`: HTML 태그 중 `name='...'` 속성을 기준으로 찾겠다
- ex. `<input t ype='password' name='password'>`

In [4]:
userId = 'user id'
input_id = driver.find_element(By.NAME, 'username')
input_id.clear()
input_id.send_keys(userId)

userPw = 'user password'
input_pw = driver.find_element(By.NAME, 'password')
input_pw.clear()
input_pw.send_keys(userPw)
input_pw.submit()

**원하는 검색어(해시태그)를 입력받아 해당 검색결과 페이지로 링크**
- 페이지가 로드되는 데 시간이 소요될 수 있어 3초의 시간 지연 부여

In [5]:
word = input('검색어를 입력하세요: ')   #'연남동맛집' 검색
url = 'https://www.instagram.com/explore/search/keyword/?q=%23' + str(word)
driver.get(url)
time.sleep(3)

In [6]:
driver.find_elements(By.CSS_SELECTOR, 'div._aagw')[0].click()   #첫 번째 게시물 클릭
time.sleep(3)

post_url = []
post_url.append(driver.current_url) #현재(첫 번째) 게시물 url

def move_next(driver):
    nxt = driver.find_element(By.CSS_SELECTOR, 'div._aaqg._aaqh')
    nxt.click()   #다음 게시물로 이동
    time.sleep(3)
    now_url = driver.current_url    #이동한 게시물 url
    post_url.append(now_url)

target = 9  #수집할 게시물 개수

for i in range(target):
    try:
        move_next(driver)   #target만큼 게시물의 url 모으기
        time.sleep(3)
    except:
        time.sleep(2)
        move_next(driver)
        time.sleep(3)

In [7]:
results = []    #수집한 데이터값 담을 리스트

for post in post_url:
    url = post
    driver.get(url) #게시물 url로 이동

    data = get_content(driver)  #크롤링
    results.append(data)
    time.sleep(3)

len(results)

10

In [8]:
results_df = pd.DataFrame(results)
results_df.columns = ['id', 'content', 'tags', 'date', 'like', 'img_urls']

results_df

#trendy_stay랑 bimp.taste는 해시태그가 댓글에 있음음

,id,content,tags,date,like,img_urls
0,seoulhotple,[먹부림하기 좋은 연남동찐맛집 36곳 대공개💗]비쥬얼부터 맛있어버린 핫한 맛집들로 ...,"[#연남동맛집, #연남동맛집추천, #연남동맛집투어, #연남동데이트, #연남동데이트추...",2023-03-31,3789,[https://scontent-ssn1-1.cdninstagram.com/v/t5...
1,seoul_trends,요즘 날씨 안가면 유죄라는연남동 봄맛집 털어왔다~~🌸🩷 비주얼부터 맛까지아무거나 골...,"[#연남동, #연남동맛집, #연남맛집, #연남동카페, #홍대맛집, #야장맛집]",2025-04-23,2400,[https://scontent-ssn1-1.cdninstagram.com/v/t5...
2,hong_sool,마포구 13년차 거주민이 말아주는💚연남 데이트코스 17💚n번째 재방문한 찐만 슬쩍 ...,"[#연남동맛집추천, #연남동맛집, #홍대맛집, #연남카페, #홍대핫플]",2025-04-30,683,[https://scontent-ssn1-1.cdninstagram.com/v/t3...
3,trendy_stay,"연남동에 이렇게 감각 있는 곳들이 많아?🤔한 번 발을 들이면 두 번, 세 번 찾게 ...",[],2025-02-27,1573,[https://scontent-ssn1-1.cdninstagram.com/v/t5...
4,or__and,연남동. 필러맞은듯한 낙지만취급한다는 미친낙지볶음집. 쫀득함이 머리울릴정도였어요. ...,"[#오얼앤드_연남동홍대, #낙지낚찌📌서울, #마포구맛집, #홍대맛집, #연남동맛집,...",2022-05-18,5만,[https://scontent-ssn1-1.cdninstagram.com/v/t5...
5,all.about.seoul.trip,✨요즘 사람들 연남동 가는 이유12✨돌고돌아 연남동만한 맛집동네 없다;;;이름에 '...,"[#연남동맛집, #연남동카페, #연남동핫플, #연남동데이트, #연남동]",2024-07-10,2093,[https://scontent-ssn1-1.cdninstagram.com/v/t3...
6,seoul_thehotple,"서울에서 오코노미야끼 제일 잘하는 곳을 소개한다.연남동 끝자락, 오코노미야끼 맛집 ...","[#소점, #더핫플_연남동, #서울맛집, #연남동맛집, #서울가볼만한곳, #서울데이...",2022-04-27,1.5만,[https://scontent-ssn1-1.cdninstagram.com/v/t5...
7,mukit_list,[먹킷📝] 이거 하나보면 끝 메모하자✏먹킷이 후기찾아보고 만든‘연남동 맛집 종결판...,"[#연남동맛집_종결판32연남동, #연남동맛집, #연남동카페, #연남동, #연남동빵집...",2022-12-08,1.8만,[https://scontent-ssn1-1.cdninstagram.com/v/t5...
8,blimp.taste,"야장보다 조용하지만, 야외 좌석에서 같은 분위기를 느낄 수 있는 맛집 8곳을 알려드...",[],2025-04-16,1484,[https://scontent-ssn1-1.cdninstagram.com/v/t5...
9,all.about.hongdae.yeonnam,여기 먹어보면 스윙스에게 고마울껄요?(๑>؂<๑)۶스윙스가 샤라웃한 연남동 수제만두...,"[#연교서울, #연남동맛집]",2025-03-31,638,[https://scontent-ssn1-1.cdninstagram.com/v/t3...
